# BTS 모델(자체 개발 모델)의 성능 측정

In [1]:
from content_based_rec import content_based_filtering
import pandas as pd
from tqdm import tqdm

인기있는 영화와 MovieLens 데이터에서 겹치는 영화를 찾아낸다.  
-> 인기있는 영화의 interaction 데이터를 얻기위해

In [2]:
mv_df = pd.read_csv('/opt/ml/input/final_project/Content_based/230129_Popular_movie_1192_cwj.csv')
ml_df = pd.read_csv('/opt/ml/input/final_project/data/ml-25m/ratings.csv')

merged_df = pd.merge(ml_df,mv_df,how='inner',on='movieId')

merged_df = merged_df.drop(['rating_x', 'timestamp', 'Contents', 'ko_title',
       'contents_year', 'country', 'ko_genre', 'ko_plot', 'rating_y', 'npop',
       'genres', 'plot', 'directors', 'actors'],axis=1)

In [3]:
mask = merged_df.userId.isin(merged_df.userId.value_counts()[merged_df.userId.value_counts() > 350].index)
interactions_df = merged_df[mask]

interactions_df = interactions_df.sort_values('userId')

In [9]:
# specify the start and end row indices
start_index = 0
end_index = 80

# group the DataFrame by the 'userId' column
grouped = interactions_df.groupby("userId")

# extract the specified rows for each group into a list of DataFrames
result = [group.iloc[start_index:end_index] for _, group in grouped]

# concatenate the list of DataFrames into a single DataFrame
train_df = pd.concat(result)

# specify the start and end row indices
start_index = 80
end_index = 140 #100

# # group the DataFrame by the 'userId' column
# grouped = interactions_df.groupby("userId")

# extract the specified rows for each group into a list of DataFrames
result = [group.iloc[start_index:] for _, group in grouped]

# concatenate the list of DataFrames into a single DataFrame
test_df = pd.concat(result)


HitRate@10을 구한다.  
80개의 interaction을 입력하고 10개의 추천리스트를 만든다.  
추천 리스트에서 입력으로 넣지 않은 유저-영화 상호작용한 데이터가 있으면 Hit이다.

In [5]:
cnt = 0
ref = []
rslt = {}
pbar = tqdm(list(zip(train_df.userId, train_df.movieId)))
for i,j in pbar:
    cnt+=1
    ref.append(j)

    if cnt%10 == 0:
        rslt[i] = content_based_filtering(ref,10)
        ref = []

100%|██████████| 48880/48880 [2:37:23<00:00,  5.18it/s]  


In [10]:
test_dict = {}
for i,j in zip(test_df.userId, test_df.movieId):
    try:
        test_dict[i].append(j)
    except KeyError:
        test_dict[i] = [j]

In [11]:
cnt = 0
for i in test_dict:
    for j in rslt[i]:
        if j[0] in test_dict[i]:
            cnt+=1
            break
print(cnt)

433


In [13]:
print(len(test_df))
print(len(test_dict))
cnt/len(test_dict) # HitRate@10

212415
611


0.7086743044189853